In [3]:
import pandas as pd 

import yfinance as yf 

import geocoder

In [17]:


# Read CSV File and start wirth line two as Header
nikkei_companies = pd.read_csv("Users/jan/Techlabs_WS_Team3/01_data/01_raw/N225_companies.csv", sep=";")





In [14]:
# Function to return adress from yahoo finance based on the ticker number 
def get_address_and_city(ticker):
    try:
        company = yf.Ticker(ticker)
        address = company.info.get('address1', 'Adresse nicht verfügbar')
        city = company.info.get('city', 'Stadt nicht verfügbar')
        return address, city
    except Exception as e:
        print(f"Fehler beim Abrufen der Daten für {ticker}: {e}")
        return 'Fehler beim Abrufen', 'Fehler beim Abrufen'


# add new columns "Adress" & "City" to dataframe 
nikkei_companies[['Address', 'City']] = nikkei_companies['Ticker'].apply(lambda ticker: pd.Series(get_address_and_city(ticker)))



We want to display the companies on the Map: For the Visualization we need the Longitude and Latitude of the City from each Company - With Geocoder we can translate the City Longitude and Latitude data. We created an Account with Mapbox to use the services via a passkey.

In [15]:
#Function to transform city into latitude and longitude - add new columns to the dateframe (longitude and latitude)

def get_lat_lon(City):
    try:
        g = geocoder.mapbox(City, key="pk.eyJ1IjoidGVjaGxhYnMzIiwiYSI6ImNtNzBib2xyczAwZHoycnBiM2hxZ24zcngifQ.AaSHNEfc-cnR4uDdEO4gsw")  # Mapbox Geocoding
        if g.ok:
            return g.latlng[0], g.latlng[1]
        return None, None
    except Exception as e:
        print(f"Fehler bei {address}: {e}")
        return None, None

# Adressen in Latitude & Longitude umwandeln
nikkei_companies["latitude"], nikkei_companies["longitude"] = zip(*[get_lat_lon(addr) for addr in nikkei_companies["City"]])



In [16]:
nikkei_companies.head()

,Number,Ticker,Company_name,Industry,Address,City,latitude,longitude
0,1,4151.T,"KYOWA KIRIN CO., LTD.",Pharmaceuticals,Otemachi Financial City Grand Cube,Tokyo,35.688820,139.692526
1,2,4502.T,"TAKEDA PHARMACEUTICAL CO., LTD.",Pharmaceuticals,"1-1, Nihonbashi-Honcho 2-chome",Tokyo,35.688820,139.692526
2,3,4503.T,ASTELLAS PHARMA INC.,Pharmaceuticals,"2-5-1, Nihonbashi-Honcho",Tokyo,35.688820,139.692526
3,4,4506.T,"SUMITOMO PHARMA CO., LTD.",Pharmaceuticals,"6-8, Doshomachi 2-chome",Osaka,34.683595,135.500784
4,5,4507.T,"SHIONOGI & CO., LTD.",Pharmaceuticals,"1-8, Doshomachi 3-chome",Osaka,34.683595,135.500784
